## **Labeling Data**

In [2]:
# Import library pandas dan numpy
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter("ignore")


In [3]:
# Mengatur opsi tampilan agar lebar maksimum kolom menjadi 155
pd.set_option("display.max_colwidth", 155)


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Menginisialisasi tokenizer menggunakan model "afbudiman/indobert-classification"
tokenizer = AutoTokenizer.from_pretrained("afbudiman/indobert-classification")

# Menginisialisasi model untuk klasifikasi urutan menggunakan model "afbudiman/indobert-classification"
model = AutoModelForSequenceClassification.from_pretrained(
    "afbudiman/indobert-classification"
)


In [10]:
df = pd.read_csv("../dataset/INA_TweetsPPKM_Raw.csv", sep="\t")


In [11]:
df["sentiment"] = " "


### **Labeling Tweets Dataset**


In [12]:
# Melakukan iterasi pada setiap baris dataset menggunakan metode iterrows() dari DataFrame
for index, row in df.iterrows():
    tweet = row["Tweet"]

    # Menggunakan tokenizer untuk mengodekan tweet menjadi input model
    encoded_input = tokenizer.encode_plus(
        tweet,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    # Memilih perangkat yang digunakan (GPU jika tersedia, jika tidak, menggunakan CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Menggunakan model untuk melakukan inferensi pada input yang diberikan
    with torch.no_grad():
        input_ids = encoded_input["input_ids"].to(device)
        attention_mask = encoded_input["attention_mask"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Mendapatkan label yang diprediksi dari logits
    predicted_label = torch.argmax(logits, dim=1).item()

    # Mengupdate nilai kolom "sentiment" pada baris yang sedang diiterasi
    df.at[index, "sentiment"] = predicted_label

In [13]:
df.to_csv(
    "../dataset/INA_TweetsPPKM_Labeled_Pure.csv",  # Menyimpan DataFrame ke file CSV dengan nama dan path yang ditentukan
    index=False,  # Tidak menyertakan index saat menyimpan ke file
    sep="\t",  # Menggunakan "\t" sebagai pemisah antar kolom dalam file CSV
    encoding="utf-8",  # Menggunakan encoding UTF-8 untuk menyimpan teks Unicode
    header="true",  # Menyertakan header kolom saat menyimpan ke file
)

In [14]:
# Mengubah tipe data kolom "sentiment" menjadi tipe data integer
df.info()
df["sentiment"] = df["sentiment"].astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23644 entries, 0 to 23643
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       23644 non-null  object
 1   User       23644 non-null  object
 2   Tweet      23644 non-null  object
 3   sentiment  23644 non-null  object
dtypes: object(4)
memory usage: 739.0+ KB


In [16]:
# Mengelompokkan data berdasarkan kolom "sentiment" dan menghitung jumlah data pada setiap kelompok
df.groupby(["sentiment"]).count()


,Date,User,Tweet
sentiment,,,
0,1958,1958,1958
1,17706,17706,17706
2,3980,3980,3980


In [18]:
df["sentiment"] = df["sentiment"].astype(int)


In [22]:
df = df[df["sentiment"] != 1]
df["sentiment"].value_counts()


2    3980
0    1958
Name: sentiment, dtype: int64

In [23]:
df['sentiment'] = df['sentiment'].replace([2,0],[0,1])


,Date,User,Tweet,sentiment
14238,2021-07-20 09:55:44+00:00,GodEL96_,"Vitamin D sangat dibutuhkan untuk kesehatan di masa pandemi begini. ""DUIT"".\n\n#dirumahaja #PPKM #motivasi",1
6231,2021-08-24 09:49:32+00:00,inilahdotcom,"Intinya, PPKM akan terus diperpanjang sampai pademinya usai. Hanya berbeda levelnya.\n\nwah repot juga nih, tidak ada cara lain selain mengikuti aturan...",0
16283,2021-07-14 09:53:21+00:00,govoes,"sekarang sudah ditahap dimana kenalan ada yang covid, uda ga kaget lagi.\n\n#ppkm",0
13090,2021-07-24 05:23:49+00:00,aryohimself,"*brigade muslim indonesia* namanya nyatut nyatut agama, tapi ajarannya ngga dipake. Bangga betul itu yg tatoan sama cewe cepak buci gitu pose depan kam...",0
11615,2021-07-27 23:42:49+00:00,lensasehatid,"Jaga kesehatan ya, kasihan keluarga merasa sedih mikirin &amp; jagain kalian kalau sakit #prokes #5m #PPKM",0
5993,2021-08-26 12:21:55+00:00,EkaMeirly,Nih lama lama ppkm ngalahin level bon cabe :v\n#PPKM,0
19078,2021-07-05 07:51:46+00:00,GonoSantoso1,"Korban Covid terus berjatuhan dari kehari terus. Tidak ada rasa bersalah, tidak ada kata maaf, dan tidak ada rasa penyesalan karena sudah membuat kebij...",0
3924,2021-09-28 13:12:12+00:00,ArifNugroho_tpm,"@kumparan Kalo kata Bu Tejo ; Tindakan Kapolri, SOLUTIP 👍😁\n@PolriBareskrim\n@febridiansyah \n@antikorupsi \n@KPK_RI \n@mohmahfudmd \n@msaid_didu \n@f...",0
12407,2021-07-26 03:44:40+00:00,dhy_Blekokk,"PPKM = Pelan Pelan Kita Miskin, ga miskin gimana kerja stop, usaha stop mau dapat duit dari mana? #PPKM",0
13665,2021-07-22 01:03:33+00:00,seicow_store,#paketpromo #makansiang bareng nih. Top banget 😍😍. \nCus order👇\nhttps://t.co/2tP7PyrW87\n\n#ppkm #surabaya #seicow #seisapi #seiayam #gwalk #surabayah...,1


In [25]:
df.sample(5)

,Date,User,Tweet,sentiment
13626,2021-07-22 03:30:05+00:00,EGINDO_co,"Jakarta sudah berubah sejak PPKM, biasanya jalanan di ibukota selalu ada kemacetan panjang. Namun, kini terlihat sepi. #PPKM #Jakarta #covid19 https://...",0
19279,2021-07-05 02:34:18+00:00,gusnung,"@Kocobenggolo1 @RachlanNashidik Aku bosen diapusi Mas Koco, di prank 🤡 terus, hoax wae #ppkm #PakPresidenKapanMundur https://t.co/Ih2fIdu03i",0
11128,2021-07-30 02:05:20+00:00,FathuRo92586941,"Diet sukses setelah #PPKM, badan lincah tiap hari muter cari utang ke temen dan saudara",1
15553,2021-07-16 07:40:44+00:00,MisbahWiyono,Ternyata efek #PPKM memberikan kesempatan Bapak nonton #ikatancinta .\n\nDisisi lain #PPKMDarurat mengurangi Kesempatan Pedagang dalam berdagang.\n\nSt...,0
15870,2021-07-15 09:00:00+00:00,bolaskorcom,"Di masa pandemi seperti sekarang ini, kita semua dituntut untuk selalu menjaga kesehatan agar tetap fit. Salah satu cara terbaik untuk mejaga kebugaran...",1


In [24]:
df["sentiment"].value_counts()


0    3980
1    1958
Name: sentiment, dtype: int64

In [26]:
df.to_csv(
    "../dataset/INA_TweetsPPKM_Labeled_PosNeg.csv",  # Menyimpan DataFrame ke file CSV dengan nama dan path yang ditentukan
    index=False,  # Tidak menyertakan index saat menyimpan ke file
    sep="\t",  # Menggunakan "\t" sebagai pemisah antar kolom dalam file CSV
    encoding="utf-8",  # Menggunakan encoding UTF-8 untuk menyimpan teks Unicode
    header="true",  # Menyertakan header kolom saat menyimpan ke file
)


### **Prune Dataset To 10.000**

- **cut the number of datasets to be used from 20 thousand to 10 thousand**


- **<p>code di bawah berikut berfungsi untuk merubah label pada dataset yang telah dilabeli sebelumnya dan mengatur ukuran dataset sebelumnya ke jumlah baru yang diinginkan pada dataset baru di bawah ini</p>**

- **<p>label : 1 (positive) & 0 (negative)</p>**


In [27]:
# Membaca file Excel menggunakan fungsi read_excel() dari pandas
df = pd.read_excel(
    "../dataset/INA_TweetsPPKM_Labeled_V1.xlsx", sheet_name="tweets_labeled"
)
df.head()


,Date,User,Tweet,Sentiment
0,2022-03-26 12:51:44,CendekiaDelapan,"Karena stimulasi untuk #anakberkebutuhankhusus tidak optimal jika dilakukan secara online.\n. \nUntuk memenuhi kebutuhan tersebut, #RumahCendekia8 teta...",2
1,2022-03-23 01:10:09,ViantAntony,Ye ampun mbh anda seorang Wapres loh.&amp;Gelar anda seorang kiyai @Kiyai_MarufAmin Knp anda terus membebani rakyatnya dengan cara2 licik sperti ini.N ...,2
2,2022-03-22 21:10:19,mtaufikmJKT48,Mulaik kumat lagi. 😀 #ppkm https://t.co/MNDSPHJU69,2
3,2022-03-22 15:43:24,PunditPintar,"Welcome PPKM bau bau naik level nih, gimana nih mang\n#ppkm",2
4,2022-03-22 15:30:34,PYI_Indonesia,"Sesungguhnya Allah tidak menciptakan kita, hambaNya ini, sebagai makhluk yang lemah lagi mudah berputus asa. Kita sebagai manusia diciptakan Allah seba...",2


In [28]:
df["Sentiment"].value_counts()


0    9482
1    7661
2    3900
Name: Sentiment, dtype: int64

In [29]:
# Membuang baris dalam DataFrame yang memiliki nilai "Sentiment" sama dengan 1
df = df[df["Sentiment"] != 1]

# Menghitung jumlah data pada setiap nilai "Sentiment"
df["Sentiment"].value_counts()


0    9482
2    3900
Name: Sentiment, dtype: int64

In [30]:
# Mengubah nilai "Sentiment" yang sama dengan 0 menjadi 1 menggunakan loc indexing
df.loc[df["Sentiment"] == 0, "Sentiment"] = 1


In [31]:
# Mengubah nilai "Sentiment" yang sama dengan 2 menjadi 0 menggunakan loc indexing
df.loc[df["Sentiment"] == 2, "Sentiment"] = 0


In [10]:
# Menghapus baris secara acak dengan nilai 1 pada kolom "Sentiment"
# Jumlah baris yang dihapus adalah selisih antara jumlah kategori sentiment 1 dengan 6100
amount_value = (6100 - 9482) * (-1)

# Menggunakan fungsi np.random.choice untuk memilih baris secara acak dengan nilai 1 pada kolom "Sentiment"
# Jumlah baris yang dihapus disimpan dalam variabel amount_value
# replace=False digunakan untuk memastikan bahwa baris yang dipilih tidak diganti (tidak digunakan kembali dalam pemilihan berikutnya)
random_remove = np.random.choice(
    df[df["Sentiment"] == 1].index, size=amount_value, replace=False
)

# Menghapus baris dengan menggunakan metode drop() pada DataFrame
df = df.drop(random_remove)


In [11]:
# Menampilkan jumlah nilai pada kolom "Sentiment"
print(df["Sentiment"].value_counts())

# Menampilkan jumlah baris dan kolom pada DataFrame
print(df.shape)


1    6100
0    3900
Name: Sentiment, dtype: int64
(10000, 4)


In [12]:
# Menyimpan DataFrame ke dalam file CSV
# File CSV disimpan dengan nama "../dataset/INA_TweetsPPKM_Labeled_FIX.csv"
# Menggunakan encoding "utf-8" untuk mendukung karakter unicode
# Menggunakan separator "\t" (tab) sebagai pemisah antara kolom-kolom dalam file CSV
# Tidak menyertakan index baris dalam file CSV dengan parameter index=False
# Menyertakan header kolom dalam file CSV dengan parameter header=True
df.to_csv(
    "../dataset/INA_TweetsPPKM_Labeled_FIX.csv",
    encoding="utf-8",
    sep="\t",
    index=False,
    header=True,
)
